https://www.philipzucker.com/compose_datalog/

Bdddbbdd
Or using SDD?

datakanren should use tries


CLP using z3 guards
Bottom up CLP https://en.wikipedia.org/wiki/Constraint_logic_programming#Bottom-up_evaluation
A(0).
A(X):-X>0.
A(X):-X=Y+1, A(Y).
fomrulog
Generation loops



revive snakelog for benchmarks


Negation - Fancy forall negation. Where did we talk about that? arntzenius?
Aggregates

LAttice / semiring
https://duckdb.org/2025/05/23/using-key.html
https://sqlite.org/lang_conflict.html

provenance?

Egglog / SLOG via 
`==` is at the same level as <= ?

extension of env

inject comments into.
Tuple together DatalogQuery

class DatalogQuery():
    q : str
    def 

env.pretty
edge(x,y) & edge(y,z)


class Env[A]:
    # Gamma |- A
    value : A 
    cols
    code


greatest fixpoint for deleting rules. DELETE FROM insterad of INSERT INTO. The delta is reversed?
Still insert into new


Monotonic graph rewriting.


Postgres has more powerful recusrive ctes? https://www.postgresql.org/docs/current/queries-with.html#QUERIES-WITH-RECURSIVE
https://news.ycombinator.com/item?id=45035618  https://vadimtropashko.wordpress.com/%e2%80%9csql-design-patterns%e2%80%9d-book/about/ some interesting tree and graph encodings. path strings

In [1]:
import psycopg2
conn = psycopg2.connect("dbname=test user=postgres")
conn.close()

OperationalError: connection to server on socket "/var/run/postgresql/.s.PGSQL.5432" failed: No such file or directory
	Is the server running locally and accepting connections on that socket?


You know, honestly the pyuthon impl is more flexible for prototyping. I'm not going to be making big stuff.

In [ ]:
from typing import Callable
type Var = str
type Env = dict[Var, object]
type DB = dict[str, set[tuple(object,...)]]
type Query = Callable[[DB], list[Env]]
from dataclasses import dataclass

@dataclass(frozen=True)
class Var():
    name : str

def reldecl(name: str) -> Callable[..., Query]:
    def rel(*args) -> Query:
        def res(db: DB) -> list[Env]:
            R = db[name]
            envs = []
            for row in R:
                env = {}
                for arg, val in zip(args, row):
                    if isinstance(arg, Var):
                        if arg in env and env[arg] != val:
                            break
                        env[arg] = val
                    else:
                        if arg != val:
                            break
                else:
                    envs.append(env)
            return envs
        return res
    return rel

def conj(*queries: Query) -> Query:
    def res(db : DB) -> list[Env]:
        envs = [{}]
        for query in queries:
            R = query(db)
            newenvs = []        
            for env1 in R:
                for env2 in envs:
                    if all(env1[k] == env2[k] for k in set(env1.keys()) & set(env2.keys())):
                        newenvs.append(env1 | env2)
            envs = newenvs
        return envs
    return res

type Action = Callable[[DB, Query], DB]

def headdecl(name : str):
    def rel(*args) -> Action:
        def res(query : Query, db : DB) -> DB:
            envs = query(db)
            Rold = db.get(name, set())
            Rnew = {tuple(env[arg] if isinstance(arg,Var) else arg for arg in args) for env in envs}
            return {**db, **{name : Rnew | Rold}}
        return res
    return rel

In [ ]:
path = set()
edge = {(1,2), (2,3), (3,4)}

def fix(*rels):
    def res(strata):
        while True:
            sizes = [len(rel) for rel in rels]
            new_rels = [set() for _ in rels]
            strata(*new_rels)
            if sizes == [len(rel) for rel in rels]:
                return
    return res

@fix(path, edge)
def strata(newpath, newedge):
    newpath |= edge
    newpath |= {(x,z) for (x,y) in edge for (y1,z) in path if y == y1}
# nah, but there is no easy way to inject the delta in there.

In [ ]:
path = set()
edge = {(1,2), (2,3), (3,4)}

def fix(*rels):
    def res(strata):
        while True:
            sizes = [len(rel) for rel in rels]
            strata(*rels)
            if sizes == [len(rel) for rel in rels]:
                return
    return res

@fix(path, edge)
def strata(path, edge):
    path |= edge
    path |= {(x,z) for (x,y) in edge for (y1,z) in path if y == y1}



    #global path, edge # choice between passing in or using global
path

class Rel():
    data: set[tuple]
    def __call__():
        pass


{(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)}

# Lattice
Use ON CONFLICT REPLACE
https://sqlite.org/lang_conflict.html
Subsumption
https://sqlite.org/lang_upsert.html

In [6]:
import sqlite3
db = sqlite3.connect(':memory:')
db.execute("CREATE TABLE edge (a NUMBER, b NUMBER, cost NUMBER, PRIMARY KEY (a,b))")
db.execute("CREATE TABLE path (a NUMBER, b NUMBER, cost NUMBER, PRIMARY KEY (a,b))")
db.execute("INSERT INTO edge VALUES (1,2,3), (2,3,4), (1,3,10)")
db.execute("INSERT INTO path SELECT * FROM edge")
print(list(db.execute("SELECT * FROM path")))
db.execute("INSERT INTO path SELECT p.a, e.b, p.cost + e.cost FROM path p, edge e WHERE p.b = e.a ON CONFLICT(a,b) DO UPDATE SET cost=min(excluded.cost, path.cost)")
print(list(db.execute("SELECT * FROM path")))

[(1, 2, 3), (2, 3, 4), (1, 3, 10)]
[(1, 2, 3), (2, 3, 4), (1, 3, 7)]


In [9]:
import sqlite3
db = sqlite3.connect(':memory:')
db.execute("CREATE TABLE edge (a NUMBER, b NUMBER, cost NUMBER, PRIMARY KEY (a,b,cost))")
db.execute("CREATE TABLE path (a NUMBER, b NUMBER, cost NUMBER, PRIMARY KEY (a,b,cost))")
db.execute("INSERT INTO edge VALUES (1,2,3), (2,3,4), (1,3,10)")
db.execute("INSERT INTO path SELECT * FROM edge")
print(list(db.execute("SELECT * FROM path")))
db.execute("INSERT OR IGNORE INTO path SELECT p.a, e.b, p.cost + e.cost FROM path p, edge e WHERE p.b = e.a")
print(list(db.execute("SELECT * FROM path")))
db.execute("DELETE FROM path WHERE EXISTS (SELECT 1 FROM path p2 WHERE p2.a=path.a AND p2.b=path.b AND p2.cost<path.cost)")
print(list(db.execute("SELECT * FROM path")))

[(1, 2, 3), (2, 3, 4), (1, 3, 10)]
[(1, 2, 3), (2, 3, 4), (1, 3, 10), (1, 3, 7)]
[(1, 2, 3), (2, 3, 4), (1, 3, 7)]


# Dynamic Stratification
Exampels that need flexible relation

Kind of like seminaive.
a :- b,c,not d.  
if frontier > d, then do it.
if d > frontier then save minimal set.
if delta = 0, move frontier to minimal set.
(set delta := frontier ?)

loop:
delta
swaparooni
if delta = 0
    swaparooni of frontier



Or perhaps we could mash frontier into a massive query using all rules.


In resolution, we are trying to show that something is unsatisfiable.
We do this by trying to generate a model.
Ordered resolution picks an ordering on models. If there is no best model, then there is no model. This cuts down on search space.

positive Datalog programs have a unique minimal model. Negation goes into ASP, with multiple stable models.
If we pick an ordering, we can again have a minimal model (ASP optimization?)

So stratified datalog infers a (partial) ordering amongst relations. We can have a finer ordering relation though, we just need to give it.


If we want to use negation but with a generic Hilog style rel2 predicate
```
rel1
rel2("edge", A,B) :- 
rel2("path", )


```
This is a move of static data to dynamic data, so it makes sense we might need another styatic to dynamic.
The relation ordering becomes data orderings a la LPO.

Well foundedness of the ordering is unneccesary.
Is totality of the ordering unnecessary?

Maybe stating a well foundedness could be used to make nonterminating datalog terminating again. Some intermixing of the datalog termination argumeny

Really cool thesis. https://www.repository.cam.ac.uk/bitstreams/6ef091ab-d041-4baa-93d3-f6c9bbb16f94/download  Furthering Datalog in the pursuit of program analysis Joseph Owen Sutherland Isaacs
Assign kind of a "sealing" relation. seal(x) > rel(x) ?

Perfect model semantics
Stefano Ceri, Georg Gottlob, and Letizia Tanca. “Extensions of Pure Datalog”. In:
Logic Programming and Databases (1990).

Closed sets is an example. Huh. So maybe this is a solution to extensional vs intensional sets?



# Why Datalog?
What is the point of all this?
https://github.com/souffle-lang/souffle/tree/master/tests/example
My junk in notes
examples in ascent? Flix?
https://github.com/HarvardPL/formulog/tree/master/examples Hmm. Not that many

https://github.com/s-arash/ascent/tree/master/ascent/examples




# Bdddbbdd
